In [ ]:
import pandas as pd
import numpy as np
from data_master import DataMaster
from utils import func
from scipy.stats import norm
master = DataMaster()
from utils.func import center
from quantstats.stats import sharpe
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
market_features = pd.read_parquet('data/US/market_features.pq')
fundamental_features = pd.read_parquet('data/US/fundamental_features.pq')
P = pd.read_parquet('data/US/test_adv_table.pq')

In [ ]:
GICS = pd.read_parquet('data/US/GICS.pq')

In [ ]:
all_features= market_features.join(fundamental_features,how = 'outer')
# del fundamental_features 

In [ ]:
START = '2019'
all_months = pd.to_datetime(P.index.strftime('%Y-%m')).tolist()
trading_months =  pd.to_datetime(P.loc[START:].index.strftime('%Y-%m')).unique().tolist()
trading_months

In [ ]:
month = trading_months[-1]
start_train = month - pd.DateOffset(months=1)
end_train = month - pd.DateOffset(days=1)
end_test = month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
training_data = all_features.sort_index(level=0).loc[start_train:end_train]
P_training = P.loc[start_train:end_train]
selected_tickers = P_training[P_training].iloc[-1].dropna().index.tolist()
print(start_train,month,end_train,end_test)

In [ ]:
market_features.columns.tolist()
NOT_USE = ['norm_daily_return',
           'daily_returns',
            'daily_vol',
            'close',
            'srs',
            'month_of_year',
            'year',
            'date',
            'target_returns',
            'alpha_daily_returns',
            'MVE',
            'CASH']

In [ ]:

features = training_data.drop(not_use,axis=1).columns.tolist()
scaled_training_data = []
for column in features:
    training_data_column = training_data[column].unstack().reindex_like(P_training)[selected_tickers].ffill().fillna(0)
    training_data_column = training_data_column.rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf)
    training_data_column = training_data_column.resample('M').mean()
    training_data_column = pd.DataFrame(training_data_column.stack(),columns = [column])
    scaled_training_data.append(training_data_column)
    

In [ ]:
from sklearn.decomposition import PCA
test = pd.concat(scaled_training_data,axis=1, join = 'outer').fillna(0)
test
pca = PCA(n_components=25)
pca.fit(test)
test_PCA = pca.fit_transform(test)
test_PCA = test.copy() #pd.DataFrame(test_PCA,index = test.index)
tickers = test_PCA.index.get_level_values(1).unique()
sz = len(test_PCA.index.get_level_values(0).unique())
n_ts =  len(test_PCA.index.get_level_values(1).unique())
d = len(test_PCA.columns.tolist())
test_PCA = test_PCA.swaplevel().sort_index(level=0)
test_PCA_array = test_PCA.values.reshape(n_ts,sz,d)


In [ ]:
from tslearn.generators import random_walks
from tslearn.clustering import TimeSeriesKMeans, KernelKMeans
km_dba = TimeSeriesKMeans(n_clusters=50, metric="dtw", max_iter=20,
                          max_iter_barycenter=20,
                          random_state=0,verbose=0).fit(test_PCA_array)
km_dba.cluster_centers_.shape
stock_assigment = km_dba.predict(test_PCA_array)
clusters = pd.DataFrame(stock_assigment,columns = ["cluster"],index = tickers)


In [ ]:
def centerize(x):
    x = x.rank(axis=1,pct=True,ascending=False).clip(0.001,0.999).apply(norm.ppf)
    x = x.sub(x.mean(1),0)
    return x
metrics = market_features["norm_monthly_return"].unstack().reindex_like(P)[selected_tickers]
metrics = metrics.loc[month:end_test]
metrics = metrics.groupby(clusters["cluster"],axis=1).apply(centerize).droplevel(0,axis=1)
metrics


In [ ]:
def centerize(x):
    x = x.rank(axis=1,pct=True,ascending=False).clip(0.001,0.999).apply(norm.ppf)
    x = x.sub(x.mean(1),0)
    return x

In [ ]:
from sklearn.decomposition import PCA
from tslearn.generators import random_walks
from tslearn.clustering import TimeSeriesKMeans, KernelKMeans
pnls = []
signals =[]
clusters_all = {}
START = '2017'
all_months = pd.to_datetime(P.index.strftime('%Y-%m')).tolist()
trading_months =  pd.to_datetime(P.loc[START:].index.strftime('%Y-%m')).unique().tolist()
SIGNAL_BASE =  market_features["norm_daily_return"].unstack().reindex_like(P)


In [ ]:
R = market_features['daily_returns'].unstack().reindex_like(P)

In [ ]:
# Use range(start, stop, step) to iterate over every fourth element
for i in tqdm(range(0, len(trading_months), 1)):
    month = trading_months[i]
    if month in list(clusters_all.keys()):
        print(month,'done')
        pass
    else :
        start_train = month - pd.DateOffset(months=1)
        end_train = month - pd.DateOffset(days=1)
        end_test = month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
        training_data = all_features.sort_index(level=0).loc[start_train:end_train]
        P_training = P.loc[start_train:end_train]
        selected_tickers = P_training[P_training].iloc[-1].dropna().index.tolist()
        features = training_data.drop(not_use,axis=1).columns.tolist()
        scaled_training_data = []
        for column in features:
            training_data_column = training_data[column].unstack().reindex_like(P_training)[selected_tickers].ffill().fillna(0)
            training_data_column = training_data_column.rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf)
            training_data_column = training_data_column.resample('M').mean()
            training_data_column = pd.DataFrame(training_data_column.stack(),columns = [column])
            scaled_training_data.append(training_data_column)
        test = pd.concat(scaled_training_data,axis=1, join = 'outer').fillna(0)
        pca = PCA(n_components=25)
        # pca.fit(test)
        test_PCA = test #pca.fit_transform(test)
        test_PCA = pd.DataFrame(test_PCA,index = test.index)
        tickers = test_PCA.index.get_level_values(1).unique()
        sz = len(test_PCA.index.get_level_values(0).unique())
        n_ts =  len(test_PCA.index.get_level_values(1).unique())
        d = len(test_PCA.columns.tolist())
        test_PCA = test_PCA.swaplevel().sort_index(level=0)
        test_PCA_array = test_PCA.values.reshape(n_ts,sz,d)
        km_dba = TimeSeriesKMeans(n_clusters=50, metric="dtw", max_iter=10,
                                max_iter_barycenter=10,
                                random_state=0,verbose=0).fit(test_PCA_array)
        stock_assigment = km_dba.predict(test_PCA_array)
        clusters = pd.DataFrame(stock_assigment,columns = ["cluster"],index = tickers)
        selected_tickers = clusters.index.tolist()
        clusters_all[month] = clusters
        signal = SIGNAL_BASE[selected_tickers].loc[month:end_test].copy()
        signal = signal.groupby(clusters["cluster"],axis=1).apply(centerize).droplevel(0,axis=1)
        returns = R[selected_tickers].loc[month:end_test]
        signal = signal.div(signal.abs().sum(1),0)
        PNL = (signal.shift()*returns).dropna(axis=0,how = 'all').sum(1)
        pnls.append(PNL)
        signals.append(signal)
        sr = sharpe(PNL)
        turnover = signal.fillna(0).diff().abs().sum(1).mean()
        PNL.cumsum().plot()
        plt.show()
        all_pnl = pd.concat(pnls)
        all_pnl.cumsum().plot()
        plt.show()
        print(pd.Series({
                'Sharpe':sr,
                'total sharpe': sharpe(all_pnl),
                'Daily Turnover (%)':turnover *100,
                'biais (bp)':PNL.mean()*1e4,
                'Sharpe since 2021' : sharpe(PNL.loc['2021':]),
                'biais  since 2021 (bp)':PNL.loc['2022':].mean()*1e4}).round(1))


In [ ]:
R = market_features['daily_returns'].unstack().reindex_like(P)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

In [ ]:
done_months = list(clusters_all.keys())
signal_features = {}
all_pnls = []
feature_source = market_features
for feature in tqdm(list(feature_source)):
    all_signal = []
    metric = feature_source[feature].unstack().reindex_like(P)[P]
    for month in (done_months) :
        grp_cluster = clusters_all[month]
        end_test = month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
        signal_month =  metric.loc[month: end_test][selected_tickers]
        selected_tickers = grp_cluster.index.tolist()
        signal_month = signal_month.groupby(grp_cluster["cluster"],axis=1).apply(centerize).droplevel(0,axis=1)
        signal_month = signal_month.div(signal_month.abs().sum(1),0)
        all_signal.append(signal_month.dropna(axis=0,how='all'))
    all_signal = pd.concat(all_signal,axis=0)
    signal_features[feature] = all_signal
    PNL = (all_signal.shift()*R).dropna(axis=0,how = 'all').sum(1)
    PNL.name = feature
    all_pnls.append(PNL)
    # sr = sharpe(PNL)
    # turnover = all_signal.fillna(0).diff().abs().sum(1).mean()
    # PNL.cumsum().plot()
    # plt.title(feature + str(sr))
    # plt.show()
    # print(pd.Series({'Sharpe':sr,
    #     'Daily Turnover (%)':turnover *100,
    #     'biais (bp)':PNL.mean()*1e4,
    #     'Sharpe since 2022' : sharpe(PNL.loc['2022':]),
    #     'biais  since 2022 (bp)':PNL.loc['2022':].mean()*1e4}).round(1))

    

In [ ]:
good = pd.concat(all_pnls,axis=1).apply(sharpe).abs().sort_values(ascending=False).head(10).index.tolist()
pd.concat(all_pnls,axis=1)[good].drop('target_returns',axis=1).cumsum().plot()

In [ ]:
done_months = list(clusters_all.keys())
signal_features = {}
feature_source = market_features
for feature in tqdm(feature_source.columns.tolist()):
    all_signal = []
    metric = feature_source[feature].unstack().reindex_like(P)
    for month in (done_months[-39:]) :
        grp_cluster = clusters_all[month]
        end_test = month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
        signal_month =  metric.loc[month: end_test][selected_tickers]
        selected_tickers = grp_cluster.index.tolist()
        signal_month = pd.DataFrame().reindex_like(P).loc[month:end_test]
        for cluster in grp_cluster['cluster'].unique().tolist():
            tickers = clusters[clusters==cluster].dropna().index.tolist()
            alpha = metric.loc[month:end_test].mean(1)
            for ticker in tickers:
                signal_month[ticker] = alpha
        signal_month = signal_month.rank(axis=1,pct=True,ascending=True).clip(0.001,0.99).apply(norm.ppf)
        signal_month = signal_month.sub(signal_month.mean(1),0)
        signal_month = signal_month.div(signal_month.abs().sum(1),0)
        all_signal.append(signal_month.dropna(axis=0,how='all'))
    all_signal = pd.concat(all_signal,axis=0)
    signal_features[feature] = all_signal
    PNL = (all_signal.shift()*R).dropna(axis=0,how = 'all').sum(1)
    sr = sharpe(PNL)
    turnover = signal.fillna(0).diff().abs().sum(1).mean()
    PNL.cumsum().plot()
    plt.title(feature + str(sr))
    plt.show()

    print(pd.Series({'Sharpe':sr,
                    'total sharpe': sharpe(all_pnl),
        'Daily Turnover (%)':turnover *100,
        'biais (bp)':PNL.mean()*1e4,
        'Sharpe since 2022' : sharpe(PNL.loc['2022':]),
        'biais  since 2022 (bp)':PNL.loc['2022':].mean()*1e4}).round(1))

    

In [ ]:
all_signals = pd.concat(signals).drop_duplicates(keep='first')
# all_signals = all_signals.rank(ascending=True,pct=True,axis=1).clip(0.001,0.999).apply(norm.ppf)
# # all_signals = all_signals[all_signals.abs()>0.5]
# all_signals = all_signals.sub(all_signals.mean(1),0)
# all_signals= all_signals.div(all_signals.abs().sum(1),0)
all_signals= all_signals
R = market_features['daily_returns'].unstack().reindex_like(P)  
pnl = (all_signals.shift()*R).dropna(axis=0,how = 'all').sum(1)
pnl.cumsum().plot()
